In [1]:
#func.py
import oracledb as od
import pandas as pd
import numpy as np
import config
from tqdm import tqdm
import ast
import re
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from scipy.linalg import svd

In [2]:
def load_recipe(n = 166511):
    od.init_oracle_client(lib_dir=r"C:\Program Files\Oracle\instantclient_21_12") # db connection
    conn = od.connect(user = config.DB_CONFIG['user'], password = config.DB_CONFIG['password'],  dsn = config.DB_CONFIG['dsn'])
    exe = conn.cursor()
    exe.execute(f'select * from recipe_table where rownum <= {n}')
    row = exe.fetchall() # row 불러오기
    column_name = exe.description # column 불러오기
    columns=[]
    for i in column_name:
        columns.append(i[0])
    result = pd.DataFrame(row, columns=columns) # row, column을 pandas DataFrame으로 나타내기
    result.rename(mapper=str.lower, axis='columns', inplace=True)
    conn.close()
    return result

func = load_recipe()
func

,recipe_url,recipe_title,recipe_summary1,recipe_summary2,recipe_summary3,recipe_ingredients,recipe_step,recipe_tag,recipe_comment,recipe_view,recipe_category_type,recipe_category_method,recipe_category_situation,row_cnt
0,http://www.10000recipe.com/recipe/6879748,순두부랑 잘어울리는 간장양념장 레시피 *,4인분,15분 이내,아무나,"{'재료': ['대파1/5대', '간장5T', '고춧가루1t', '소금2꼬집', '...","['#1 재료를 준비해주세요 !', '#2 대파는 다져서 준비해주세요.부추가 있으면...","['순두부', '양념장', '간장양념장', '순두부 양념장', '순두부간장양념장',...",0.0,7937.0,양념/소스/잼,기타,일상,94003
1,http://www.10000recipe.com/recipe/6922184,다 쏟은 기력 되살려줄 낙지찜 ★,2인분,30분 이내,중급,"{'필수재료': ['콩나물200g', '낙지300g', '바지락200g', '양파1...","['#1 양파는 채 썰고 미나리는 5cm 길이로 썰고 대파와 고추는 어슷 썬다.',...","['낙지찜', '낙지', '낙지레시피', '낙지찜 양념', '매콤', '저녁메뉴',...",0.0,5858.0,메인반찬,찜,손님접대,94004
2,http://www.10000recipe.com/recipe/6907492,수미네반찬 감자채볶음,2인분,15분 이내,초급,"{'재료': ['감자큰거1개', '양파작은거1개', '스팸3조각', '청양고추1개'...","['#1 재료 씻고 깍고 꺼내 준비', '#2 양파어슷썰고', '#3 스팸 적당두께...","['감자채볶음', '감자', '수미네반찬', '초간단', '수미네반찬레시피']",0.0,3143.0,밑반찬,볶음,초스피드,94005
3,http://www.10000recipe.com/recipe/6833147,"계란오이볶음 :: 백주부레시피 , 오이반찬",4인분,10분 이내,초급,"{'재료': ['계란5개', '오이1개', '청양고추2개(생략가능)', '파10cm...",['#1 청양고추와 대파를 송송 썰어둡니다. 파기름을 내려면 파가 필요한데 아무리 ...,"['집밥백선생', '백주부', '백주부레시피', '초간단', '다이어트']",0.0,7593.0,밑반찬,볶음,일상,94006
4,http://www.10000recipe.com/recipe/6909699,삼겹살김밥 으로 푸짐한 도시락 만들기♡,4인분,30분 이내,초급,"{'재료': ['삼겹살200g', '느타리버섯50g', '깻잎9장', '김밥속재료1...","['#1 파 1뿌리를 파채썰어서 양념준비합니다.', '#2 고기 짝꿍 파절임만들기^...","['삼겹살김밥', '간단요리', '도시락', '푸짐한한끼', '김밥만들기', '초보...",0.0,2962.0,밥/죽/떡,기타,도시락,94007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166506,http://www.10000recipe.com/recipe/6929107,"몽글 몽글 쉽게 만드는 ""계란찜""",2인분,15분 이내,초급,"{'재료': ['계란6개', '간장1수저', '설탕1수저', '미림1수저', '물1...","['#1 계란 6개를 준비한다', '#2 설탕 1수저와 소금 1/2수저를 넣어준다'...",[],0.0,3714.0,밑반찬,찜,일상,9235
166507,http://www.10000recipe.com/recipe/6961108,새콤달콤한 아삭함이 매력적인 시원한 오이피클 레시피,4인분,20분 이내,아무나,"{'재료': ['오이1개', '소금1/2T'], '소스': ['물1종이컵', '설탕...",['#1 손에 소금 1/2T를 얹어 오이 표면을 골고루 비벼주세요. 소금으로 오이표...,"['오이', '오이피클', '오이피클만들기', '오이피클만드는방법', '수제오이피클...",0.0,1210.0,밑반찬,절임,일상,9236
166508,http://www.10000recipe.com/recipe/6964170,초 간단 오징어실채 무침~*,3인분,5분 이내,아무나,"{'재료': ['오징어실채110g'], '양념': ['간장2큰술', '참기름1.5큰...","['#1 오징어실채 탈탈 털어 볼에담고 가위로 한 두번 잘라준다', '#2 간장,고...","['간단밑반찬', '오징어실채', '마른반찬', '도시락반찬', '아이들반찬', '...",0.0,706.0,밑반찬,무침,일상,9237
166509,http://www.10000recipe.com/recipe/6964163,새로운 비법양념으로 무친 대파김치청포묵무침,2인분,10분 이내,아무나,"{'재료': ['청포묵1팩', '대파김치1/2컵', '김가루'], '양념': ['갈...","['#1 청포묵을 준비합니다', '#2 물을 끓입니다', '#3 청포묵을 썰어놓습니...","['대파김치청포묵무침', '청포묵무침', '청포묵', '묵무침', '묵요리', '한...",0.0,873.0,밑반찬,무침,일상,9238


In [4]:
def recipe_preprocessing(raw):
    data = raw.loc[raw['recipe_ingredients'].notnull()].copy()  # None 값 제거

    def clean_ingredients(ingredients):
        if ingredients is not None:
            ingredients = ingredients.replace('\\ufeff', '').replace('\\u200b', '')
        return ingredients

    def not_empty_ingredients(row):
        return row['recipe_ingredients'].strip() != '{}' # 결측치 제거

    data["recipe_ingredients"] = data["recipe_ingredients"].apply(clean_ingredients)
    data = data[data.apply(not_empty_ingredients, axis=1)]
    result = data[['recipe_title', 'recipe_ingredients']]

    title_idx = result[result['recipe_title'].isnull()].index
    del_idx = result[result['recipe_ingredients'].str.startswith('소시지')].index
    result.drop(del_idx, inplace=True)
    result.drop(title_idx, inplace=True)

    return result

func_pre = recipe_preprocessing(func)
func_pre

C:\Users\glori\AppData\Local\Temp\ipykernel_14212\3402541563.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.drop(del_idx, inplace=True)
C:\Users\glori\AppData\Local\Temp\ipykernel_14212\3402541563.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.drop(title_idx, inplace=True)


,recipe_title,recipe_ingredients
0,순두부랑 잘어울리는 간장양념장 레시피 *,"{'재료': ['대파1/5대', '간장5T', '고춧가루1t', '소금2꼬집', '..."
1,다 쏟은 기력 되살려줄 낙지찜 ★,"{'필수재료': ['콩나물200g', '낙지300g', '바지락200g', '양파1..."
2,수미네반찬 감자채볶음,"{'재료': ['감자큰거1개', '양파작은거1개', '스팸3조각', '청양고추1개'..."
3,"계란오이볶음 :: 백주부레시피 , 오이반찬","{'재료': ['계란5개', '오이1개', '청양고추2개(생략가능)', '파10cm..."
4,삼겹살김밥 으로 푸짐한 도시락 만들기♡,"{'재료': ['삼겹살200g', '느타리버섯50g', '깻잎9장', '김밥속재료1..."
...,...,...
166506,"몽글 몽글 쉽게 만드는 ""계란찜""","{'재료': ['계란6개', '간장1수저', '설탕1수저', '미림1수저', '물1..."
166507,새콤달콤한 아삭함이 매력적인 시원한 오이피클 레시피,"{'재료': ['오이1개', '소금1/2T'], '소스': ['물1종이컵', '설탕..."
166508,초 간단 오징어실채 무침~*,"{'재료': ['오징어실채110g'], '양념': ['간장2큰술', '참기름1.5큰..."
166509,새로운 비법양념으로 무친 대파김치청포묵무침,"{'재료': ['청포묵1팩', '대파김치1/2컵', '김가루'], '양념': ['갈..."


In [13]:
title_idx = func_pre[func_pre['recipe_title'].isnull()].index

In [8]:
del_idx = func_pre[func_pre['recipe_ingredients'].str.startswith('소시지')].index

In [19]:
func_pre.drop(del_idx, inplace=True)
func_pre.drop(title_idx, inplace=True)

In [20]:
func_pre
#157263 > 157258

,recipe_title,recipe_ingredients
0,만물상 5분 콩국수,"{'재료': ['메주콩5소주잔', '옥수수알1소주잔', '물2컵', '중면1인분',..."
1,아삭하고 깔끔한 오이소박이 비법 *^^*,"{'재료': ['오이10개', '부추2/3단', '쪽파6줄기', '양파1개', '천..."
2,탕수육...바삭한 탕수육에 특제 탕수육소스^^,"{'재료': ['돼지고기안심400g', '양파1/2개', '표고버섯(목이버섯)4개'..."
3,잔멸치나 새우보다 칼슘이 최고! 견과류를 넣은 뱅어포조림,"{'재료': ['뱅어포3장', '식용유1/2컵', '아몬드20알', '해바라기씨1큰..."
4,한우 모듬 초밥,"{'재료': ['한우차돌박이1/2줌', '노란파프리카1/4개', '빨간파프리카1/4..."
...,...,...
166506,일본식 닭조림 만들기,"{'재료': ['닭다리또는닭봉1팩', '감자2개', '당근1/2개', '표고버섯3개..."
166507,블랙라벨스테이크 깨쟁이요리 럭셔리도시락,"{'재료': ['블랙라벨스테이크260g', '양파1/3', '그린빈4개', '방울토..."
166508,특별한 별미 곤약조림 탱글탱글 짭쪼름 맛있는 밑반찬,"{'재료': ['통곤약600g', '물900ml', '간장60ml', '매실액10m..."
166509,오믈렛만드는법*브런치만들기,"{'계란 오믈렛 재료': ['계란2개', '소금약간', '소세지약간', '모짜렐라치..."


In [ ]:
def split_ingredient(data):
    for i in range(1, 75):
        data.loc[:, f'ingredient{i}'] = None
        data.loc[:, f'quantity{i}'] = None
        data.loc[:, f'unit{i}'] = None


    non_matching_items = {} # 패턴과 일치하지 않는 데이터를 저장할 딕셔너리

    for idx, row in tqdm(data.iterrows(), total=data.shape[0]): #tqdm으로 진행상황 확인
        if row['recipe_ingredients']:
            ingredients_dict = ast.literal_eval(row["recipe_ingredients"]) #딕셔너리 형태로 저장된 recipe_ingredients 불러오기
            ingredient_count = 1
            for category, items in ingredients_dict.items(): #category : 재료, 양념재료, items: 사과1개, 돼지고기600g
                if items:  # 아이템이 존재하는 경우
                    for item in items:
                        match = re.match(r'([가-힣a-zA-Z]+(\([가-힣a-zA-Z]+\))?|\d+[가-힣a-zA-Z]*|\([가-힣a-zA-Z]+\)[가-힣a-zA-Z]+)([\d.+/~-]*)([가-힣a-zA-Z]+|약간|조금)?', item) # 정규식
                        if match:
                            ingredient, _, quantity, unit = match.groups()

                            data.loc[idx, f'ingredient{ingredient_count}'] = ingredient
                            data.loc[idx, f'quantity{ingredient_count}'] = quantity
                            data.loc[idx, f'unit{ingredient_count}'] = unit

                            ingredient_count += 1
                        else:
                            # 패턴과 일치하지 않는 경우 딕셔너리에 추가
                            non_matching_items[idx] = item
        else:
            pass

    # 패턴과 일치하지 않는 데이터 출력 X => 날려버리기!
    for idx, item in non_matching_items.items():
        print(f'Row {idx}: {item}')
    #        recipe_ingre_pd_pre.drop([idx], axis=0, inplace=True)
    return data

func_pre_split = split_ingredient(func_pre[:78631])

In [ ]:
func_pre_split

In [ ]:
for_del = func_pre_split[func_pre_split[('ingredient1')].isnull()].index

In [ ]:
func_pre_split.drop([{for_del}], axis=0, inplace=True)

In [ ]:
func_pre_split[func_pre_split[('unit1')].isnull()]

In [ ]:
recipe_ingre_pd_pre[recipe_ingre_pd_pre[('ingredient24')].notnull()]

In [ ]:
recipe_ingre_pd_pre_del = recipe_ingre_pd_pre.copy()
recipe_ingre_pd_pre_del.drop([1853], axis = 0, inplace=True)
recipe_ingre_pd_pre_del.drop(['recipe_ingredients', 'ingredient21', 'quantity21', 'unit21', 'ingredient22', 'quantity22', 'unit22', 'ingredient23', 'quantity23', 'unit23', 'ingredient24', 'quantity24', 'unit24', 'ingredient25', 'quantity25', 'unit25', 'ingredient26', 'quantity26', 'unit26', 'ingredient27', 'quantity27', 'unit27', 'ingredient28', 'quantity28', 'unit28', 'ingredient29', 'quantity29', 'unit29'], axis = 1, inplace = True)
recipe_ingre_pd_pre_del

In [ ]:
#약간/적당량 다 빼기
units = [recipe_ingre_pd_pre_del[f'ingredient{i}'] for i in range(1, 21)]

combined_ingredients = []
for unit_tuple in zip(*units):
    for unit in unit_tuple:
        if unit:  # 빈 값이 아닌 경우에만 추가
            combined_units.append(unit)

# 중복 제거
unique_units = list(set(combined_units))

from collections import Counter

unit_counts = Counter(combined_units)
for unit, count in unit_counts.items():
    print(f"{unit}: {count}번")